In [ ]:
import matplotlib as mpl
import matplotlib.pyplot as plt
%matplotlib inline
import numpy as np
import sklearn
import pandas as pd
import os
import sys
import time
import tensorflow as tf

from tensorflow import keras

print(tf.__version__)
print(sys.version_info)
for module in mpl, np, pd, sklearn, tf, keras:
    print(module.__name__, module.__version__)

In [ ]:
# data source download:
# wget https://storage.googleapis.com/tf-datasets/titanic/train.csv
# wget https://storage.googleapis.com/tf-datasets/titanic/eval.csv

In [ ]:
train_file = "../temp/data/train.csv"
eval_file = "../temp/data/eval.csv"

train_df = pd.read_csv(train_file)
eval_df = pd.read_csv(eval_file)

print(train_df.head())
print(eval_df.head())

In [ ]:
y_train = train_df.pop('survived')
y_eval = eval_df.pop('survived')

print(train_df.head())
print(y_train.head())
print(eval_df.head())
print(y_eval.head())

In [ ]:
train_df.describe()

In [ ]:
print(train_df.shape, eval_df.shape)

In [ ]:
train_df.age.hist(bins = 20)

In [ ]:
train_df.sex.value_counts().plot(kind = 'barh')

In [ ]:
train_df['class'].value_counts().plot(kind = 'barh')

In [ ]:
pd.concat([train_df, y_train], axis = 1).groupby('sex').survived.mean().plot(kind = 'barh')

In [ ]:
categorical_columns = ['sex', 'n_siblings_spouses', 'parch', 'class', 'deck', 'embark_town', 'alone']
numeric_columns = ['age', 'fare']

feature_columns = []
for categorical_column in categorical_columns:
    vocab = train_df[categorical_column].unique()
    print(vocab)
    feature_columns.append(
        tf.feature_column.indicator_column(
            tf.feature_column.categorical_column_with_vocabulary_list(categorical_column, vocab)))

for numeric_column in numeric_columns:
    feature_columns.append(
        tf.feature_column.numeric_column(numeric_column, dtype=tf.float32))

In [ ]:
def make_dataset(data_df, label_df, epochs = 10, shuffle = True, batch_size = 32):
    dataset = tf.data.Dataset.from_tensor_slices((dict(data_df), label_df))
    if shuffle:
        datase = dataset.shuffle(10000)
    dataset = dataset.repeat(epochs).batch(batch_size)
    return dataset

In [ ]:
train_dataset = make_dataset(train_df, y_train, batch_size=5)

In [ ]:
for x, y in train_dataset.take(1):
    print(x, y)

In [ ]:
# kaeras.layers.DenseFeature
for x, y in train_dataset.take(1):
    age_column = feature_columns[7]    # keep the same
    gender_column = feature_columns[0] # 1-hot encoding
    print(keras.layers.DenseFeatures(age_column)(x).numpy())
    print(keras.layers.DenseFeatures(gender_column)(x).numpy())

In [ ]:
# kaeras.layers.DenseFeature
for x, y in train_dataset.take(1):
    print(keras.layers.DenseFeatures(feature_columns)(x).numpy())
    print(y)

In [ ]:
model = keras.models.Sequential([
        keras.layers.DenseFeatures(feature_columns),
        keras.layers.Dense(100, activation='relu'),
        keras.layers.Dense(100, activation='relu'),
        keras.layers.Dense(2, activation='softmax')
    ])
model.compile(loss='sparse_categorical_crossentropy',
             optimizer = keras.optimizers.SGD(lr=0.01),
             metrics = ['accuracy'])

In [ ]:
# 1. model.fit

train_dataset = make_dataset(train_df, y_train)
eval_dataset = make_dataset(eval_df, y_eval, epochs = 1, shuffle = False)
history = model.fit(train_dataset,
                   validation_data=eval_dataset,
                   steps_per_epoch=20,
                   validation_steps=8,
                   epochs=100)

In [ ]:
model = keras.models.Sequential([
        keras.layers.DenseFeatures(feature_columns),
        keras.layers.Dense(100, activation='relu'),
        keras.layers.Dense(100, activation='relu'),
        keras.layers.Dense(2, activation='softmax')
    ])
model.compile(loss='sparse_categorical_crossentropy',
             optimizer = keras.optimizers.SGD(lr=0.01),
             metrics = ['accuracy'])

In [ ]:
# 2. model -> estimator -> train
estimator = keras.estimator.model_to_estimator(model)
# input_fn can be:
# 1. funciton, which returns a.(features, labels), b.dataset -> (feature, label)
estimator.train(input_fn=lambda: make_dataset(train_df, y_train, epochs=10))
